In [1]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [1]:
import numpy as np
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop

from sumo_grid_simulation.grid_simulation import Simulator

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [2]:
simulator = Simulator()

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [3]:
def user_function_time_loss(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray """
    result = []
    print(X)
    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = int(edgeLength),
            numberOfLanes = int(numLanes),
            accel         = accel
        )
        result.append(s['timeLoss'])
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model (GP)

Our surrogate model is our emulator. In this case, a gaussian process. I think that emukit handles categorical/discrete inputs using one-hot encodings.

#### Model Inputs

In [5]:
gridSize = ContinuousParameter('gridSize', min_value=3, max_value=25)
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=1, max_value=25)
maxSpeed = ContinuousParameter('maxSpeed', min_value=1, max_value=25)
edgeLength = ContinuousParameter('edgeLength', min_value=2, max_value=250)
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3,4])
accel = ContinuousParameter('accel', 0.2, 10)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numberOfLanes, accel])

#### Initialize Model / Emulator (GP)

In [6]:
design = RandomDesign(parameter_space)  # initialize with random points
num_data_points = 10
init_X = design.get_samples(num_data_points)
init_Y = user_function_time_loss(init_X)

[[ 15.07301289   1.98353131  22.62517987 162.12772275   4.
    7.15171609]
 [  8.85289705  15.84590847   5.36980788 137.21952764   4.
    7.21996077]
 [ 10.03239852   4.50754156  17.0547725   61.25773676   3.
    7.43640295]
 [  8.94157628  20.4729841    4.40364854 234.93636703   1.
    7.03497571]
 [  6.75046592  12.53254475   5.0060201  116.80839185   2.
    9.26260887]
 [ 14.7438653   21.11457771  23.78893208 170.21070316   4.
    9.73512645]
 [ 12.76892073  20.04400091   7.48239245 186.74216417   3.
    2.915376  ]
 [ 22.78773654   9.2482371    1.39325413 185.15766495   4.
    7.23390271]
 [  4.75883662   1.75115132  19.75706724  55.72665967   2.
    0.89619117]
 [  3.5095607   23.42706265   4.55726009  41.4986788    3.
    3.12628421]]
Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysong

Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation

In [7]:
init_X.shape, init_Y.shape

((10, 6), (10, 1))

In [8]:
init_X, init_Y

(array([[ 15.07301289,   1.98353131,  22.62517987, 162.12772275,
           4.        ,   7.15171609],
        [  8.85289705,  15.84590847,   5.36980788, 137.21952764,
           4.        ,   7.21996077],
        [ 10.03239852,   4.50754156,  17.0547725 ,  61.25773676,
           3.        ,   7.43640295],
        [  8.94157628,  20.4729841 ,   4.40364854, 234.93636703,
           1.        ,   7.03497571],
        [  6.75046592,  12.53254475,   5.0060201 , 116.80839185,
           2.        ,   9.26260887],
        [ 14.7438653 ,  21.11457771,  23.78893208, 170.21070316,
           4.        ,   9.73512645],
        [ 12.76892073,  20.04400091,   7.48239245, 186.74216417,
           3.        ,   2.915376  ],
        [ 22.78773654,   9.2482371 ,   1.39325413, 185.15766495,
           4.        ,   7.23390271],
        [  4.75883662,   1.75115132,  19.75706724,  55.72665967,
           2.        ,   0.89619117],
        [  3.5095607 ,  23.42706265,   4.55726009,  41.4986788 ,
        

In [9]:
emulator = GPy.models.GPRegression(init_X, init_Y)
emukit_model = GPyModelWrapper(emulator)
emulator

GP_regression.,value,constraints,priors
rbf.variance,1.0,+ve,
rbf.lengthscale,1.0,+ve,
Gaussian_noise.variance,1.0,+ve,


### Optimization

#### Acquisition Function

In [10]:
model_variance = ModelVariance(model=emukit_model)

#### Experimental Design

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model,
    space = parameter_space,
    acquisition = model_variance,
    batch_size = 1
)

In [12]:
ed_loop.run_loop(user_function_time_loss, 150)

Optimization restart 1/1, f = 70.47921643407683
[[ 15.19220884  18.89779499   6.19947531 165.52303985   2.
    5.91080032]]
Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarout

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 77.08478872977899
[[ 7.80524181  9.34994746 15.52808304 96.00400239  4.          4.45493238]]
Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarout

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 83.60784236582968
[[ 24.78334912  19.04630394  18.56478727 197.25065362   4.
    2.4450965 ]]
Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarout

...

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 1090.854021156868
[[ 3.  1.  1.  2.  4. 10.]]
Success.


calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursewor

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/1, f = 1096.8045579696538
[[  3.   1.   1. 250.   4.  10.]]
Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Cou

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


### Evaluate

#### Validate on init points

In [21]:
pred_Y, pred_Y_variance = emukit_model.predict(init_X)
mse = np.sqrt(np.mean((pred_Y-init_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  1.1247524857947935e-10


In [22]:
pred_Y

array([[346.44],
       [ 64.71],
       [ 37.59],
       [142.22],
       [ 49.69],
       [ 30.62],
       [ 32.97],
       [789.23],
       [ 25.48],
       [  7.11]])

In [20]:
init_Y

array([[346.44],
       [ 64.71],
       [ 37.59],
       [142.22],
       [ 49.69],
       [ 30.62],
       [ 32.97],
       [789.23],
       [ 25.48],
       [  7.11]])

#### Test on 10 randomly chosen points

In [13]:
design = RandomDesign(parameter_space)  # initialize with random points
num_data_points = 10
test_X = design.get_samples(num_data_points)
test_Y = user_function_time_loss(test_X)
pred_Y, pred_Y_variance = emukit_model.predict(test_X)

[[ 18.1557065    9.12428222  24.92405944  11.45844227   1.
    6.77365423]
 [ 15.89638014   6.07019038   4.57656477  72.06162579   2.
    5.42312269]
 [ 19.98974798   4.37581374  24.33778989 172.22587872   3.
    6.21870606]
 [ 11.328174     7.74227395   7.74693695 237.72957631   1.
    9.98320769]
 [  7.65715866  21.71753093  13.95268199 168.34128953   3.
    6.88639092]
 [ 20.33567444  12.94538418   3.98119761  90.31075225   2.
    4.84705051]
 [ 15.81481182   9.7330655   21.27931198  73.88789736   3.
    2.6620262 ]
 [ 23.4886678    2.15407706  21.14933399  65.04910418   4.
    9.89067377]
 [  6.52684648   4.31910695  18.34546411 143.57252883   3.
    7.48633289]
 [ 16.93314639  10.78143097  21.96106567  39.49743927   1.
    0.57932673]]
Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysong

Success.

calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/grid.net.xml -r /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.trips.xml --ignore-errors --begin 0.0 --end 3600.0 --no-step-log --additional-files /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh.add.xml --no-warnings -o /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation/simulation_input_files/veh_passenger.rou.xml
Success.
calling /usr/local/Cellar/sumo/1.8.0/share/sumo/bin/duarouter -n /Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu/sumo_grid_simulation

Avg error:  14590.332586718278


In [17]:
mse = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Root mean squared error: ', mse)

Root mean squared error:  120.79044907076998


In [18]:
pred_Y

array([[ 16.91759537],
       [ 98.20844845],
       [199.90890357],
       [451.0270485 ],
       [-28.17991988],
       [128.2077359 ],
       [ 55.37131318],
       [114.40630853],
       [221.84629298],
       [-11.62902689]])

In [19]:
test_Y

array([[  8.93],
       [ 67.04],
       [203.99],
       [113.52],
       [ 20.03],
       [124.3 ],
       [ 17.3 ],
       [166.18],
       [ 69.86],
       [ 25.4 ]])